In [1]:
#imports
from requests.auth import HTTPBasicAuth
import requests
import pymongo
from dotenv import load_dotenv
import os

In [2]:
# settings.py
load_dotenv()
clockify_api_key = os.getenv("CLOCKIFY_API_KEY")
mongo_conn_string = os.getenv("MONGO_CONN")

In [3]:
headers = {"Accept": "application/json", 'x-api-key': clockify_api_key}

## Get Tags

In [4]:
tags_url = "https://api.clockify.me/api/v1/workspaces/5f25e65aadbc875dd9032dee/tags"
tags = requests.get(tags_url, headers=headers)

In [5]:
tags = tags.json()

In [6]:
for t in tags:
    t['tag_id'] = t['id']
    t['tag_name'] = t['name']
    del t['id']
    del t['name']
    del t['workspaceId']
    del t['archived']

In [7]:
tags[0]

{'tag_id': '5f39bcd7d62fa00f8cf356ec', 'tag_name': 'Data Science'}

## Get Projects

In [8]:
projects_url = "https://api.clockify.me/api/v1/workspaces/5f25e65aadbc875dd9032dee/projects"
projects = requests.get(projects_url, headers=headers)

In [9]:
projects = projects.json()

In [10]:
for p in projects:
    p['project_id'] = p['id']
    p['project'] = p['name']
    p['project_duration'] = p['duration']
    del p['id']
    del p['duration']
    del p['name']
    del p['hourlyRate']
    del p['clientId']
    del p['workspaceId']
    del p['billable']
    del p['memberships']
    del p['color']
    del p['estimate']
    del p['archived']
    del p['clientName']
    del p['note']
    del p['template']
    del p['public']

In [11]:
projects[0]

{'project_id': '5f272b89e89e8809e381b2f7',
 'project': 'Blog',
 'project_duration': 'PT29H30M'}

## Get Entries

In [12]:
entries_url = "https://api.clockify.me/api/v1/workspaces/5f25e65aadbc875dd9032dee/user/5f25e659adbc875dd9032ded/time-entries"
entries = requests.get(entries_url, headers=headers)

In [13]:
entries = entries.json()

In [14]:
for e in entries:
    e['entry_id'] = e['id']
    e['entry_start'] = e['timeInterval']['start']
    e['entry_finish'] = e['timeInterval']['end']
    e['entry_duration'] = e['timeInterval']['duration']
    e['entry'] = e['description']
    e['tags'] = []
    del e['id']
    del e['timeInterval']
    del e['description']
    del e['customFieldValues']
    del e['userId']
    del e['billable']
    del e['taskId'] 
    del e['workspaceId']
    del e['isLocked']

In [15]:
for e in entries:
    for entry_tag in e['tagIds']:
        for t in tags:
            if t['tag_id'] == entry_tag:
                e['tags'].append(t['tag_name'])
        

In [16]:
for e in entries:
    del e['tagIds']

In [17]:
entries[0]

{'projectId': '5f39f190803448349949e6df',
 'entry_id': '5f56c3971c7fea640944fe79',
 'entry_start': '2020-09-07T23:20:00Z',
 'entry_finish': '2020-09-07T23:30:00Z',
 'entry_duration': 'PT10M',
 'entry': 'Meditation',
 'tags': ['Meditation']}

## Format Results

In [18]:
#for entry in entries:
#    entry["description"]
import pandas as pd

# join entries and projects
df_e1 = pd.DataFrame(entries).set_index('projectId')
df_projects = pd.DataFrame(projects).set_index('project_id')
df_entries = df_e1.merge(df_projects, left_index=True, right_index=True)


df_entries.set_index(['entry_start','entry_finish'], inplace=True, drop=True)

#df.T.to_dict()


for col in df_entries.columns: 
    print(col) 


entry_id
entry_duration
entry
tags
project
project_duration


In [19]:
new_entries = df_entries['entry_id'].count()

In [20]:
client = pymongo.MongoClient(mongo_conn_string)
db = client['jim_blog']
clockify = db["clockify"]
saved_entries = db.clockify.count_documents({})

In [21]:
if new_entries > saved_entries:
    d_count = clockify.delete_many({})
    print(d_count.deleted_count, " clockify documents deleted.")
    
    data_dict = df_entries.to_dict("records")
    clockify.insert_many(data_dict)
    print(db.clockify.count_documents({}), " clockify documents created")

else:
    print('nothing to do')


nothing to do
